### Logistic Regression

In [ ]:
import numpy as np

class LogisticRegression:
    def __init__(self,lr,iters):
        self.lr = lr
        self.iters = iters
        self.weights = None
        self.bias = None
    
    def sigmod(self,z):
        # 1 / (1 + e^(-z))
        return 1 / (1 + np.exp(-z)) 
    